In [ ]:
# REF: https://www.kaggle.com/dansbecker/exercise-modeling-from-scratch

In [1]:
import numpy as np
import pandas as pd
pd.options.display.max_columns = 12
pd.options.display.float_format = '{:.1f}'.format

import tensorflow as tf

# Disable warnings in Anaconda
import warnings
warnings.simplefilter('ignore')

import matplotlib.pyplot as plt
# We will use the Seaborn library
import seaborn as sns
sns.set()
# Graphics in SVG format are more sharp and legible
%config InlineBackend.figure_format = 'svg'
# Increase the default plot size
from pylab import rcParams
rcParams['figure.figsize'] = 8, 8

# We will display plots right inside Jupyter Notebook
%matplotlib inline


/usr/local/miniconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Conv2D, Dropout

img_rows, img_cols = 28, 28
num_classes = 10

In [4]:
# def data_prep(raw):
#     out_y = tf.keras.utils.to_categorical(raw.label, num_classes)
    
#     num_imgs = raw.shape[0]
#     x_as_array = raw.values[:, 1:]
#     x_shaped_array = x_as_array.reshape(num_images, img_rows, img_cols, 1)
#     out_x = x_shaped_array / 255
#     return out_x, out_y

def prep_data(raw, train_size, val_size):
    y = raw[:, 0]
    out_y = tf.keras.utils.to_categorical(y, num_classes)
    
    x = raw[:, 1:]
    num_images = raw.shape[0]
    out_x = x.reshape(num_images, img_rows, img_cols, 1)
    out_x = out_x / 255
    return out_x, out_y

In [8]:
train_file = "_db/fashion-mnist_train.csv"
df_train = np.loadtxt(train_file, skiprows=1, delimiter=',')
x, y = prep_data(df_train, train_size=50000, val_size=5000)

print(x.shape)
print(y.shape)

(60000, 28, 28, 1)
(60000, 10)


###### Specify Model

**STEPS:**

1. 建立一個 Sequential model. 稱作 fashion_model.

2. 加入三個 Conv2D 層至 fashion_model. 讓每層有十二個 filters, 一個 kernel_size of 3 以及一個 relu activation. 需要指定第一層 Conv2D shape. 輸入的 shape 是 (img_rows, img_cols, 1).

3. 最後一個 Conv2D 後，加一 Flatten 層

4. 接著，加一個有100 神經元的 Dense 層於 Flatten 層之後.

5. 最後，加入預測層至 fashion_model. 這是一個 Dense 層，其中第一個參數為 num_classes. 第二個參數為 activation function  (softmax).

In [10]:
fashion_model = Sequential()
fashion_model.add(Conv2D(12, 
                         kernel_size=(3,3),
                         activation='relu',
                         input_shape=(img_rows, img_cols, 1)))
fashion_model.add(Conv2D(12, kernel_size=(3, 3), activation='relu'))
fashion_model.add(Conv2D(12, kernel_size=(3, 3), activation='relu'))
fashion_model.add(Flatten())
fashion_model.add(Dense(100))
fashion_model.add(Dense(num_classes, activation='softmax'))

fashion_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_3 (Conv2D)            (None, 26, 26, 12)        120       
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 24, 24, 12)        1308      
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 22, 22, 12)        1308      
_________________________________________________________________
flatten_1 (Flatten)          (None, 5808)              0         
_________________________________________________________________
dense_2 (Dense)              (None, 100)               580900    
_________________________________________________________________
dense_3 (Dense)              (None, 10)                1010      
Total params: 584,646
Trainable params: 584,646
Non-trainable params: 0
_________________________________________________________________


# Compile Model

Run the command fashion_model.compile. Specify the following arguments:

1. loss = keras.losses.categorical_crossentropy

2. optimizer = 'adam'

3. metrics = ['accuracy']

In [12]:
fashion_model.compile(loss=tf.keras.losses.categorical_crossentropy,
                     optimizer='adam',
                     metrics=['accuracy'])

# Fit Model

Run the command fashion_model.fit. The arguments you will use are

1. The first two are arguments are the data used to fit the model, which are x and y respectively.

2. batch_size = 100

3. epochs = 4

4. validation_split = 0.2

When you run this command, you can watch your model start improving. You will see validation accuracies after each epoch.

In [13]:
fashion_model.fit(x, y,
                 batch_size=100,
                 epochs=4,
                 validation_split=0.2)

Train on 48000 samples, validate on 12000 samples
Epoch 1/4
48000/48000 [==============================] - 44s 913us/step - loss: 0.3851 - acc: 0.8625 - val_loss: 0.3482 - val_acc: 0.8783
Epoch 2/4
48000/48000 [==============================] - 44s 908us/step - loss: 0.3018 - acc: 0.8900 - val_loss: 0.3045 - val_acc: 0.8924
Epoch 3/4
48000/48000 [==============================] - 45s 937us/step - loss: 0.2575 - acc: 0.9077 - val_loss: 0.3099 - val_acc: 0.8857
Epoch 4/4
48000/48000 [==============================] - 44s 909us/step - loss: 0.2311 - acc: 0.9164 - val_loss: 0.2910 - val_acc: 0.8988
